In [10]:
import json

with open("mondial_full_dataset.json", "r") as f:
    data = json.load(f)
    
for item in data["dataset"]:
    print(item)
    break
    
dataset = data["dataset"]

{'id': '1', 'type': 'simple', 'question': 'What is the area of Thailand?', 'query': "SELECT max(area) FROM mondial_country where name  = 'Thailand'", 'keywords': '', 'answer': [{'uri': '/results_table/1.csv'}], 'tables': ['country']}


## Limpando o dataset, para temos apenas itens que participam de mais de um banco da federação

In [23]:
from schemas.mondial_federated_schema import MONDIAL_SOCIAL, MONDIAL_POlITICS, MONDIAL_GEO, MONDIAL_ECONOMY

# pega a lista de tabelas para cada banco da federação
def get_table_names(schema):
    table_names = []
    for table in schema.split("CREATE TABLE "):
        table_name = table.split("\n")[0]
        table_names.append(table_name.lower())
    return table_names[1:] # exlui o primeiro que é vazio.

# pega a lista de tabelas para cada banco da federação
economy_db_tables_names = get_table_names(MONDIAL_ECONOMY)
geo_db_tables_names = get_table_names(MONDIAL_GEO)
social_db_tables_names = get_table_names(MONDIAL_SOCIAL)
politics_db_tables_names = get_table_names(MONDIAL_POlITICS)

print("Economy tables:", economy_db_tables_names)
print("--------------------")
print("Geography tables:", geo_db_tables_names)
print("--------------------")
print("Social tables:", social_db_tables_names)
print("--------------------")
print("Politics tables:", politics_db_tables_names)

Economy tables: ['economy', 'population']
--------------------
Geography tables: ['country', 'riverthrough', 'encompasses', 'continent', 'city', 'province', 'mountain', 'desert', 'island', 'lake', 'sea', 'river', 'airport', 'geo_mountain', 'geo_desert', 'geo_island', 'geo_river', 'geo_sea', 'geo_lake', 'geo_source', 'geo_estuary', 'located', 'locatedon', 'islandin', 'mountainonisland', 'lakeonisland', 'riveronisland', 'mergeswith']
--------------------
Social tables: ['religion', 'ethnicgroup', 'language', 'sublanguage', 'countrylocalname', 'countryothername', 'provincelocalname', 'provinceothername', 'citylocalname', 'cityothername', 'countrypops', 'provpops', 'citypops']
--------------------
Politics tables: ['politics', 'borders', 'organization', 'ismember']


In [27]:
new_dataset = []

# Passa por todos itens e verifica se tem mais de um banco da federação envolvido nas tabelas do item.
for item in dataset:
    if len(item["tables"]) > 1:
        tables_involved = set()
        for table in item["tables"]:
            if table in economy_db_tables_names:
                tables_involved.add("economy")
            elif table in geo_db_tables_names:
                tables_involved.add("geography")
            elif table in social_db_tables_names:
                tables_involved.add("social")
            elif table in politics_db_tables_names:
                tables_involved.add("politics")
        
            if len(tables_involved) > 1:
                new_dataset.append(item)
                break

print(len(dataset))
print(len(new_dataset))

100
31


## Disso vemos que do dataset original, apenas 31 exemplos podem ser testados em cima da nossa federação criada. Mas vamos trabalhar com eles.

In [26]:
# salva em um novo .json

with open("federated_mondial_dataset.json", "w") as f:
    json.dump({"dataset": new_dataset}, f)